# Modèle - Titanic

- Connaitre le path du fichier

In [ ]:
!pwd

- Importation de la librairie Pandas

In [ ]:
import pandas as pd

- Importation des données

In [ ]:
df = pd.read_csv('datas/titanic/train.csv')

- Afficher le tableau de données

In [ ]:
df.head()

- Vérifier les valeur null

In [ ]:
df.isnull()

- Afficher les types de données du tableau

In [ ]:
df.dtypes

- Changer le type d'une colonne

In [ ]:
df["Fare"] = pd.to_numeric(df["Fare"])
df.dtypes

- Nettoyage des données

In [ ]:
df1 = df.drop(['Name','Ticket','Cabin','PassengerId','Embarked'],axis = 1)
# df.drop(["Name","Sex","Ticket","Cabin","Embarked"], axis=1, inplace=True)

- Rassembler 2 colonnes

In [ ]:
pd.concat([pd.DataFrame(df.dtypes), pd.DataFrame(df.isnull().sum())], axis=1)

- Génération du model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df1['Sex'] = df['Sex'].map({'male': 1, 'female': 0})
df1

In [ ]:
df2=df1.dropna()
df2

- Affichage de chaque types de données

In [ ]:
df2.dtypes

In [ ]:
y = df2.Survived
x = df2.drop(['Survived'], axis = 1)

x_train,x_test, y_train, y_test = train_test_split(x,y,test_size = .2, random_state = 42)

In [ ]:
print (x_train.shape,x_test.shape)
print (df2.shape)

In [ ]:
print (y_train.sample(10))

- Importation de la forêt aléatoire avec sélection d'arbres basée sur Monte Carlo (RF-TSMC)
- Cela consiste à utiliser l'approche Random Forest pour la classification multiclasse en utilisant une sélection d'arbre dynamique basée sur Monte Carlo.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

- Création d'un premier modèle :
    - n_estimators = nombre d'arbres
    - random_state = le hazard à 23
    - max_features = le nombre de colonne sélectionner par itération au hazard

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, random_state = 23, max_features = 2)

In [ ]:
rf.fit(X=x_train, y=y_train)

In [ ]:
#Train value
print ('train : ')
print (rf.score(x_train,y_train))

#Test value
print ('test : ')
print(rf.score(x_test,y_test))

In [ ]:
def print_score(learn_obj, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test):
	print("Score train : ", learn_obj.score(x_train, y_train,)*100, "%")
	print("Score test : ", learn_obj.score(x_test, y_test,)*100, "%")

print_score(rf)

# Explicabilité

- Chaque colonne elle prend en compte à 8% Pclass, 25% le Sex etc, ainsi c'est l'importance des données en entrer 

In [ ]:
pd.DataFrame(rf.feature_importances_*100,x_train.columns, columns=['Importance en %'])

#pd.DataFrame(rf.feature_importances_, x_train.columns)

- Importation de SHAP :
    - SHAP (SHapley Additive exPlanations) est une approche unifiée pour expliquer la sortie de tout modèle d'apprentissage automatique.

In [ ]:
import shap

In [ ]:
shpv = shap.TreeExplainer(rf).shap_values(x_train)

In [ ]:
shap.summary_plot(shpv, x_train)

# Viz

- Importation du générateur de rapport de profile

In [ ]:
from pandas_profiling import ProfileReport

- Génèrer un rapport de profil à partir d'un pandas des données

In [ ]:
profile = ProfileReport(df)

- Afficher un rapport de profil d'un pandas des données

In [ ]:
print(profile)
# ---à vérifier---

- Exporter un rapport de profil d'un pandas des données

In [ ]:
profile.to_file('model_titanic.html')

In [ ]:
profile.to_notebook_iframe()

# Graphique explication influence sexe dans mortalité

In [ ]:
import plotly.express as px

female_train = df[df["Sex"] == 'female']
fig = px.scatter(x=female_train['Age'], y=female_train['Fare'], color=female_train['Survived'])
fig.show()

male_train = df[df["Sex"] == 'male']
fig = px.scatter(x=male_train['Age'], y=male_train['Fare'], color=male_train['Survived'])
fig.show()

In [ ]:
# En utilisant une fonction
def graph_scatter_age(dd):
	fig = px.scatter(x=dd['Age'], y=dd['Fare'], color=dd['Survived'])
	fig.show()
# graph_scatter_age(dd[dd["Sex"] == 'female'])

In [ ]:
df.plot.box()

In [ ]:
df.Fare.plot.box()

In [ ]:
df.Fare.mean()

In [ ]:
from sklearn.linear_model import RidgeClassifier

rc = RidgeClassifier()
rc.fit(X=x_train, y=y_train)

print_score(rc)